In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo

  Activating project at `~/daniel/ocrl-piano/Julia_fmincon_tests`
  No Changes to `~/daniel/ocrl-piano/Julia_fmincon_tests/Project.toml`
  No Changes to `~/daniel/ocrl-piano/Julia_fmincon_tests/Manifest.toml`


In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
# import piano model
model = load_model("piano_with_one_shadow_hand/scene.xml")


MuJoCo Model with 117 bodies and 114 joints

Model info

In [4]:
println("number of geoms: ",model.ngeom)
finger_indices = [115,116,123,124,131,132,141,142,150,151]
step!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
finger_geom_indices = [116,124,132,142,151]
"""
index finger:
    name:
        rh_shadow_hand//unnamed_geom_25
        rh_shadow_hand//unnamed_geom_26
    index:
        115
        116
    range:
        x:
        y:-0.6-0.5
        z:0-0.3

middle finger:
    name:
        rh_shadow_hand//unnamed_geom_33
        rh_shadow_hand//unnamed_geom_34
    index:
        123
        124
ring finger:
    name:
        rh_shadow_hand//unnamed_geom_41
        rh_shadow_hand//unnamed_geom_42
    index:
        131
        132
pinky finger:
    name:
        rh_shadow_hand//unnamed_geom_51
        rh_shadow_hand//unnamed_geom_52
    index:
        141
        142
thumb:
    name:
        rh_shadow_hand//unnamed_geom_60
        rh_shadow_hand//unnamed_geom_61
    index:
        150
        151
""";

number of geoms: 152


UndefVarError: UndefVarError: `data` not defined

Visualize target finger position

In [40]:
reset!(model, data)
@show model.nq
@show model.nv
@show model.nu


qpos_target = copy(data.qpos)
qpos_target[94] = 0.5
qpos_target[95] = 0.5
qpos_target[96] = 0.5

data.qpos .= qpos_target
data.qpos[1:2] = [2, 2]
step!(model, data)
@show data.qpos
@show data.qvel
@show typeof(vec(vcat(data.qpos, data.qvel)))
@show data.ctrl
init_visualiser()
visualise!(model, data)
println()

xic = randn(model.nq + model.nv)
@show typeof(xic)
x = get_physics_state(model, data)

indices = vcat(94:96, 114+94:114+96)
@show size(x[indices])

model.nq = 114
model.nv = 114
model.nu = 22
data.qpos = [1.8953692701879468; 1.8995641930659128; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.8235953435

(6,)

## Test DIRCOL: moving index finger left right

In [14]:
function wrapped_mj_step(model, data, x, uk)
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[7:9] = uk
        # set state
        # data.qpos .= x[1:model.nq]
        # data.qvel .= x[(model.nq + 1):end]
        data.qpos[94:96] = x[1:3]
        data.qvel[94:96] = x[4:6]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= converted_uk

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    step!(model, data) 

    # return updated state k + 1
    xkp1 = zeros(model.nq + model.nv) 
    xkp1 .= get_physics_state(model, data)
    # finger_coordinates = data.geom_xpos[finger_geom_indices,:]
    
    return xkp1
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    model = params.model
    data = params.data
    Q, R, Qf = params.Q, params.R, params.Qf

    # stage cost
    cost = 0.0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]

        # @show xi-xg
        # cost += 0.5*(xi[94:96]-xg[94:96])'*Q*(xi[94:96]-xg[94:96])+0.5*ui'*R*ui
        cost += 0.5*(xi-xg)'*Q*(xi-xg)+0.5*ui'*R*ui
    end

    # terminal cost 
    xf = Z[idx.x[N]]
    # cost += 0.5*(xf[94:96]-xg[94:96])'*Qf*(xf[94:96]-xg[94:96])
    cost += 0.5*(xf-xg)'*Qf*(xf-xg)

    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = params.data
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        data.qpos[94:96] = xi[1:3]
        data.qvel[94:96] = xi[4:6]
        xip1_mujoco = wrapped_mj_step(model, data, xi, ui)
        # c[idx.c[i]] = xip1_mujoco[94:96] - xip1[94:96]

        indices = vcat(94:96, 114+94:114+96)
        c[idx.c[i]] = xip1_mujoco[indices] - xip1
    end
    # println(typeof(c))
    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    data = params.data

    con_1 = Z[idx.x[1]] - xic
    con_2 = Z[idx.x[N]] - xg

    return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    # return [con_1; con_2]
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    return zeros(eltype(Z), 0)
    # return inequality_constraints
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [51]:

function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("piano_with_one_shadow_hand/scene.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)
    
    # initiate time and time steps
    dt = 0.005
    tf = 0.25
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost
    # Q size of full states
    # Q = diagm(ones(model.nq + model.nv))
    # R = 0.1*diagm(ones(model.nu))
    # Qf = 10*diagm(ones(model.nq + model.nv))

    Q = 1*diagm(ones(6))
    # R = 0.1*diagm(ones(model.nu))
    R = 0.1*diagm(ones(3))
    Qf = 10*diagm(ones(6))

    # initial and goal states
    xic = vec(vcat(copy(data.qpos[94:96]), copy(data.qvel[94:96])))

    # xg = vec(vcat(copy(data.qpos), copy(data.qvel)))
    # xg[94] = 0.5
    # xg[95] = 0.5
    # xg[96] = 0.5
    xg = [0.5, 0.5, 0.5, 0, 0, 0]

    # xic = randn(model.nq + model.nv)
    # xg = xic

    # indexing 
    # idx = create_idx(model.nq + model.nv, model.nu, N)
    idx = create_idx(6, 3, N)
    
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)


    # primal bounds
    # x_l = -1 * Inf * ones(idx.nz)
    x_l = -1*ones(idx.nz)
    # x_u = Inf*ones(idx.nz)
    x_u = ones(idx.nz)
    

    # inequality constraints
    # c_l = -1 * Inf * ones(3*(idx.N-1))
    # c_u = Inf * ones(3*(idx.N-1))
    c_l = zeros(0)
    c_u = zeros(0)

    # initial guess 
    z0 = 0.001*randn(idx.nz)
    z0[idx.x[1]] = xic

    # diff type
    # diff_type = :auto 
    diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-3, c_tol = 1e-3, max_iters = 25, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [52]:
X, U, t_vec, params = solve_finger_moving(verbose=true)

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:   142272
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      456
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      456
                     variables with only upper bounds:        0
Total number of equality constraints.................:      312
Total number of inequality constraints.......

([[-0.024690407683151238, -0.014141296248814574, 0.005603184127693126, 0.005905001158191563, 0.002788161461752901, 0.00165841790368812], [-0.031185873680188478, -0.014450410107794062, 0.007064856476908905, 0.19127144965869497, -0.03718881719906428, -0.05977566687330234], [-0.04143965608585348, -0.014302574890513852, 0.00932678797996845, -0.6210234294583701, -0.1035892385534599, -0.07905302668295651], [-0.048318235950185986, -0.014858366175886374, 0.011585337461464007, 0.1645654318769069, -0.2133970071768986, -0.08322517278177666], [-0.05624897689627302, -0.016413172051081223, 0.013208681697172275, -0.0778932459032603, -0.29053716876285446, -0.0869750985269777], [-0.06320178965422282, -0.015640327104328226, 0.015624116409790938, 0.0006059046220067699, 0.044962076082068606, -0.06449255331942047], [-0.0694450700110357, -0.015250308021308518, 0.017954604577731258, 0.08936185887915028, -0.0033320475564051766, -0.05460821528113044], [-0.07487461415952373, -0.014856511455255775, 0.02131847442

In [55]:
# instantiate model and data
model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)

states[:,1] = get_physics_state(model, data)

# for t in 1:(N-1)
#     data.ctrl .= zeros(model.nu)
#     # data.ctrl[7:9] = U[t][:]
#     step!(model, data)
#     states[:,t+1] = get_physics_state(model, data)
# end

for t in 1:N
    reset!(model, data)
    data.qpos[94:96] = X[t][1:3]
    data.qvel[94:96] = X[t][4:6]
    states[:,t] = get_physics_state(model, data)
end


In [56]:
init_visualiser()
visualise!(model, data, trajectories = states)

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.



2886r 6.0283676e+00 4.07e+00 2.86e+04   0.4 2.43e-01    -  1.00e+00 1.00e+00h  1
2887r 6.0572511e+00 2.85e+00 4.15e+04   0.4 4.97e-01    -  1.00e+00 1.00e+00h  1
2888r 6.0628306e+00 2.72e+00 1.74e+05   0.4 1.85e-01    -  8.86e-01 6.13e-02h  4
2889r 6.0703610e+00 2.57e+00 2.80e+05   0.4 2.05e-01    -  1.00e+00 5.62e-02h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2890r 6.1306098e+00 1.53e+00 5.22e+05   0.4 1.86e-01    -  9.86e-01 5.00e-01h  2
2891r 6.1415871e+00 1.37e+00 2.03e+04   0.4 1.53e-01    -  1.00e+00 9.88e-02h  4
